In [1]:
from enum import Enum

# 七節　温水暖房

## 付録B　ガス温水暖房機及びガス給湯温水暖房機

In [2]:
def get_gas_cons(hs_type, temp_sw, q_out, eff_rtd, q_rtd):
    
    if q_out > 0.0:
        
        # q_body = heat loss from body, MJ/h (eq.2)
        # f_rtd = corrected_coefficient
        q_body, f_rtd = {
            'gasConv' : (240.96 * 3600 * 10**(-6), 0.985),
            'gasCondensing' : {
                '60' : (225.26 * 3600 * 10**(-6), 1.038),
                '40' : (123.74 * 3600 * 10**(-6), 1.064)
            }[str(temp_sw)]
        }[hs_type]
        
        # heat exchange efficiency (eq.3)
        heat_exchange_coeff \
            = eff_rtd * f_rtd * (q_rtd * 3600 * 10**(-6) + q_body) / (q_rtd * 3600 * 10**(-6))
        
        # gas consumption, MJ/h (eq.1)
        return (q_out + q_body) / heat_exchange_coeff

    else:
        return 0.0

In [3]:
# 式（4）
def get_E_E_hs(_E_E_hs_pmp, _E_E_hs_fan):
    return _E_E_hs_pmp + _E_E_hs_fan

In [4]:
# 式（5）
# _r_WS_hsは4章本編の話と思われる。
def get_E_E_hs_pmp(_r_WS_hs):
    _P_hs_pmp = 73 # Unit : W
    return _P_hs_pmp * _r_WS_hs * 10**(-3)

In [5]:
# 式（6）
def get_E_E_hs_fan(_E_G_hs):
    r = 0.003
    return _E_G_hs * r * 10**3 / 3600
    

In [6]:
# 式（7）
def get_Q_max_H_hs(_q_rtd_hs):
    return _q_rtd_hs * 3600 / 10**(6)


In [7]:
class HeatRadiator_MainRoom_Exist(Enum):
    Exist = 1,
    NotExist = 2

class HeatRadiator_OtherRoom_Exist(Enum):
    Exist = 1,
    NotExist = 2
    

In [8]:
# 付録H　温水暖房用熱源機の最大能力

# 暖冷房区画の床面積には、温水暖房により暖房される暖冷房区画のみを積算する。

# 主居室、その他居室の放熱器の設置有無　＊　運転方法

def get_q_max_hs(_region, _A_HCZ_i, _HeatRadiator_MainRoom_Exist, _HeatRadiator_OtherRoom_Exist, _HeatRadiator_MainRoom_Operation, _HeatRadiator_OtherRoom_Operation):
    
    _f_CT = 1.05
    
    # _q_rq_H
    if _HeatRadiator_MainRoom_Exist == HeatRadiator_MainRoom_Exist.Exist and _HeatRadiator_OtherRoom_Exist == HeatRadiator_OtherRoom_Exist.Exist:
        _HeatRadiator_Exist = 'BothRoom_Exist'
    elif _HeatRadiator_MainRoom_Exist == HeatRadiator_MainRoom_Exist.Exist and _HeatRadiator_OtherRoom_Exist == HeatRadiator_OtherRoom_Exist.NotExist:
        _HeatRadiator_Exist = 'MainRoom_Exist'
    elif _HeatRadiator_MainRoom_Exist == HeatRadiator_MainRoom_Exist.NotExist and _HeatRadiator_OtherRoom_Exist == HeatRadiator_OtherRoom_Exist.Exist:
        _HeatRadiator_Exist = 'OtherRoom_Exist'
    
    coef = {'Region1' : {'BothRoom_Exist' : 90.02, 'MainRoom_Exist' : 139.26, 'OtherRoom_Exist' : 62.28},
            'Region2' : {'BothRoom_Exist' : 77.81, 'MainRoom_Exist' : 120.65, 'OtherRoom_Exist' : 53.26},
            'Region3' : {'BothRoom_Exist' : 73.86, 'MainRoom_Exist' : 111.32, 'OtherRoom_Exist' : 83.81},
            'Region4' : {'BothRoom_Exist' : 77.74, 'MainRoom_Exist' : 118.98, 'OtherRoom_Exist' : 55.41},
            'Region5' : {'BothRoom_Exist' : 83.24, 'MainRoom_Exist' : 126.56, 'OtherRoom_Exist' : 59.43},
            'Region6' : {'BothRoom_Exist' : 69.76, 'MainRoom_Exist' : 106.48, 'OtherRoom_Exist' : 49.93},
            'Region7' : {'BothRoom_Exist' : 74.66, 'MainRoom_Exist' : 112.91, 'OtherRoom_Exist' : 53.48},
            'Region8' : {'BothRoom_Exist' : None,  'MainRoom_Exist' : None,   'OtherRoom_Exist' : None}
    }
    
    _q_rq_H = coef[_region][_HeatRadiator_Exist]
    
    # _f_CI
      # 主たる居室の運転方法、その他の居室の運転方法
    _f_CI = {'Continuous'   : {'Continuous' : 1.0 , 'Intermittent' : 1.0 , 'NotExist' : 1.0 },
             'Intermittent' : {'Continuous' : 2.25, 'Intermittent' : 2.25, 'NotExist' : 3.03},
             'NotExist'     : {'Continuous' : 1.0 , 'Intermittent' : 1.62}
    }[_HeatRadiator_MainRoom_Operation][_HeatRadiator_OtherRoom_Operation]
    
    print(_q_rq_H , sum(_A_HCZ_i) , _f_CT , _f_CI, sep=", ")
    
    return _q_rq_H * sum(_A_HCZ_i) * _f_CT * _f_CI
    
   

#### Example

In [9]:
get_q_max_hs(_region = 'Region2', _A_HCZ_i = (29.81,0), _HeatRadiator_MainRoom_Exist = HeatRadiator_MainRoom_Exist.NotExist, _HeatRadiator_OtherRoom_Exist = HeatRadiator_OtherRoom_Exist.Exist, _HeatRadiator_MainRoom_Operation = 'Intermittent', _HeatRadiator_OtherRoom_Operation = 'Intermittent')

53.26, 29.81, 1.05, 2.25


3750.8954175000003

In [10]:
_q_rtd_hs = get_q_max_hs(_region = 'Region2', _A_HCZ_i = (29.81,0, 0, 0, 0), _HeatRadiator_MainRoom_Exist = HeatRadiator_MainRoom_Exist.NotExist, _HeatRadiator_OtherRoom_Exist = HeatRadiator_OtherRoom_Exist.Exist, _HeatRadiator_MainRoom_Operation = 'NotExist', _HeatRadiator_OtherRoom_Operation = 'Intermittent')

_Q_max_H_hs = get_Q_max_H_hs(_q_rtd_hs)
_Q_max_H_hs
    

53.26, 29.81, 1.05, 1.62


9.722320922160002